In [1]:
import os
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset
import optuna

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/users/eleves-a/2024/elio.samaha/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_embeddings(file_path):
    """
    Load embeddings and labels from the pickle file and extract MatchID from the PeriodID.
    """
    print(f"Loading embeddings from {file_path}...")
    merged_df = pd.read_pickle(file_path)
    if "ID" in merged_df.columns:
        merged_df["MatchID"] = merged_df["ID"].apply(lambda x: x.split("_")[0])
    print(f"Loaded {len(merged_df)} PeriodIDs.")
    return merged_df

# Load training and test data
train_embeddings_file = "aggregated_embeddings_with_labels.pkl"
test_embeddings_file = "aggregated_embeddings_with_labels_test.pkl"

train_df = load_embeddings(train_embeddings_file)
test_df = load_embeddings(test_embeddings_file)

Loading embeddings from aggregated_embeddings_with_labels.pkl...
Loaded 2137 PeriodIDs.
Loading embeddings from aggregated_embeddings_with_labels_test.pkl...
Loaded 516 PeriodIDs.


In [3]:
def create_game_based_data_loaders(df, batch_size, sequence_length, shuffle=True):
    """
    Create DataLoaders grouped by game. Each DataLoader contains sequences of embeddings.
    """
    game_loaders = {}
    grouped = df.groupby("MatchID")

    for match_id, group in grouped:
        all_embeddings = np.vstack(group["aggregated_embedding"].values).astype(np.float32)
        all_labels = group["EventType"].values.astype(int)

        sequences = []
        sequence_labels = []

        # Create sequences of specified length
        for i in range(len(all_embeddings) - sequence_length + 1):
            seq_emb = all_embeddings[i:i + sequence_length]
            seq_label = all_labels[i + sequence_length - 1]  # Use label of last embedding in sequence
            sequences.append(seq_emb)
            sequence_labels.append(seq_label)

        if len(sequences) == 0:
            continue

        sequences = np.array(sequences)  # Shape: [num_sequences, sequence_length, input_dim]
        sequence_labels = np.array(sequence_labels)  # Shape: [num_sequences]

        dataset = TensorDataset(
            torch.tensor(sequences, dtype=torch.float32),
            torch.tensor(sequence_labels, dtype=torch.float32)
        )
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        game_loaders[match_id] = loader

    return game_loaders

In [4]:
# Splitting games into training, validation, and internal testing
all_games = train_df["MatchID"].unique()
np.random.seed(42)
np.random.shuffle(all_games)

# Reserve 3 games for internal testing
internal_test_games = all_games[:3]
train_val_games = all_games[3:]

# Create cross-validation splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_loaders = []
for train_index, val_index in kf.split(train_val_games):
    train_split = train_val_games[train_index]
    val_split = train_val_games[val_index]

    train_df_cv = train_df[train_df["MatchID"].isin(train_split)]
    val_df_cv = train_df[train_df["MatchID"].isin(val_split)]

    train_game_loaders = create_game_based_data_loaders(train_df_cv, batch_size=32, sequence_length=5)
    val_game_loaders = create_game_based_data_loaders(val_df_cv, batch_size=32, sequence_length=5)
    cv_loaders.append((train_game_loaders, val_game_loaders))

# Internal test loaders
internal_test_df = train_df[train_df["MatchID"].isin(internal_test_games)]
internal_test_loaders = create_game_based_data_loaders(internal_test_df, batch_size=32, sequence_length=5)

In [5]:
class ChronologicalLSTM(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, num_layers=2, bidirectional=True, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers=num_layers, dropout=dropout,
            bidirectional=bidirectional, batch_first=True
        )
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(lstm_output_dim, 1)  # Binary classification

    def forward(self, embeddings):
        lstm_out, _ = self.lstm(embeddings)
        logits = self.fc(lstm_out[:, -1, :])  # Use the last timestep output
        return logits

In [6]:
def train_model(model, train_game_loaders, val_game_loaders, epochs, lr, device, patience=10, weight_decay=1e-5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()

    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss, train_accuracy = 0, 0
        total_train_batches = 0

        for match_id, loader in train_game_loaders.items():
            for embeddings, labels in loader:
                embeddings, labels = embeddings.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(embeddings).squeeze(-1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy()
                train_accuracy += accuracy_score(labels.cpu().numpy(), preds)
                total_train_batches += 1

        avg_train_loss = train_loss / total_train_batches
        avg_train_acc = train_accuracy / total_train_batches
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_acc:.4f}")

        # Validation loop
        model.eval()
        val_accuracy = 0
        total_val_batches = 0
        with torch.no_grad():
            for match_id, loader in val_game_loaders.items():
                for embeddings, labels in loader:
                    embeddings, labels = embeddings.to(device), labels.to(device)
                    outputs = model(embeddings).squeeze(-1)
                    preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy()
                    val_accuracy += accuracy_score(labels.cpu().numpy(), preds)
                    total_val_batches += 1

        avg_val_acc = val_accuracy / total_val_batches if total_val_batches > 0 else 0
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {avg_val_acc:.4f}")

        if avg_val_acc > best_val_acc + 1e-6:  # Slight tolerance for floating-point issues
            best_val_acc = avg_val_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    return model

In [7]:
def evaluate_model(model, test_game_loaders, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for match_id, loader in test_game_loaders.items():
            for embeddings, labels in loader:
                embeddings, labels = embeddings.to(device), labels.to(device)
                outputs = model(embeddings).squeeze(-1)
                preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [8]:
def fine_tune_model(cv_loaders, device, n_trials=20):
    def objective(trial):
        try:
            # Suggest hyperparameters
            lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
            hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
            dropout = trial.suggest_float("dropout", 0.1, 0.5)
            sequence_length = trial.suggest_int("sequence_length", 3, 10)

            # Initialize the model with trial parameters
            model = ChronologicalLSTM(
                input_dim=768,
                hidden_dim=hidden_dim,
                num_layers=2,
                bidirectional=True,
                dropout=dropout
            ).to(device)

            # Cross-validation loop
            val_accuracies = []
            for train_game_loaders, val_game_loaders in cv_loaders:
                # Recreate loaders with the suggested sequence length
                train_loaders = create_game_based_data_loaders(
                    pd.concat([train_df[train_df["MatchID"].isin(list(train_game_loaders.keys()))]]),
                    batch_size=32,
                    sequence_length=sequence_length
                )
                val_loaders = create_game_based_data_loaders(
                    pd.concat([train_df[train_df["MatchID"].isin(list(val_game_loaders.keys()))]]),
                    batch_size=32,
                    sequence_length=sequence_length
                )
                # Train and evaluate
                trained_model = train_model(model, train_loaders, val_loaders, epochs=5, lr=lr, device=device)
                val_accuracy = evaluate_model(trained_model, val_loaders, device)
                val_accuracies.append(val_accuracy)

            # Return mean validation accuracy for the trial
            mean_val_accuracy = np.mean(val_accuracies)
            print(f"Trial completed. Mean Validation Accuracy: {mean_val_accuracy:.4f}")
            return mean_val_accuracy
        except Exception as e:
            print(f"Trial failed with error: {e}")
            return None  # Explicitly return None for failed trials

    # Create Optuna study and optimize
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # Ensure at least one successful trial
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    if len(completed_trials) == 0:
        raise ValueError("No successful trials. Please check the objective function or data.")

    return study.best_params

In [17]:
def predict_for_submission(model, test_df, device):
    """
    Generate predictions for the Kaggle test set using the trained model.
    """
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        grouped = test_df.groupby("MatchID")
        for match_id, group in grouped:
            embeddings = np.vstack(group["aggregated_embedding"].values).astype(np.float32)
            embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32).to(device)

            # Add sequence length dimension for LSTM
            embeddings_tensor = embeddings_tensor.unsqueeze(1)  # Shape: [batch_size, 1, input_dim]

            outputs = model(embeddings_tensor).squeeze(-1)
            preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy()
            group["EventType"] = preds.astype(int)
            predictions.append(group[["ID", "EventType"]])
    return pd.concat(predictions)

In [10]:
# Initialize the model
initial_model = ChronologicalLSTM(input_dim=768, hidden_dim=128, num_layers=2, bidirectional=True, dropout=0.3)

# Train the model using cross-validation
for fold_idx, (train_loaders, val_loaders) in enumerate(cv_loaders):
    print(f"Training fold {fold_idx + 1}/{len(cv_loaders)}...")
    initial_model = train_model(initial_model, train_loaders, val_loaders, epochs=50, lr=1e-4, device=device)


Training fold 1/5...
Epoch 1/50, Train Loss: 0.6917, Train Accuracy: 0.5450
Epoch 1/50, Validation Accuracy: 0.5659
Epoch 2/50, Train Loss: 0.6898, Train Accuracy: 0.5455
Epoch 2/50, Validation Accuracy: 0.5625
Epoch 3/50, Train Loss: 0.6873, Train Accuracy: 0.5519
Epoch 3/50, Validation Accuracy: 0.5660
Epoch 4/50, Train Loss: 0.6885, Train Accuracy: 0.5427
Epoch 4/50, Validation Accuracy: 0.5641
Epoch 5/50, Train Loss: 0.6854, Train Accuracy: 0.5376
Epoch 5/50, Validation Accuracy: 0.5716
Epoch 6/50, Train Loss: 0.6807, Train Accuracy: 0.5358
Epoch 6/50, Validation Accuracy: 0.6065
Epoch 7/50, Train Loss: 0.6723, Train Accuracy: 0.5587
Epoch 7/50, Validation Accuracy: 0.6106
Epoch 8/50, Train Loss: 0.6582, Train Accuracy: 0.5983
Epoch 8/50, Validation Accuracy: 0.6506
Epoch 9/50, Train Loss: 0.6550, Train Accuracy: 0.6192
Epoch 9/50, Validation Accuracy: 0.6745
Epoch 10/50, Train Loss: 0.6465, Train Accuracy: 0.6300
Epoch 10/50, Validation Accuracy: 0.7033
Epoch 11/50, Train Loss: 0.

In [11]:
# Test the model on the internal test set
print("Evaluating the initial model on the internal test set...")
evaluate_model(initial_model, internal_test_loaders, device)

Evaluating the initial model on the internal test set...
Test Accuracy: 0.5187


0.5186915887850467

In [12]:
# Fine-tune the model
best_params = fine_tune_model(cv_loaders, device)

# Re-initialize the model with the best hyperparameters
fine_tuned_model = ChronologicalLSTM(
    input_dim=768,
    hidden_dim=best_params["hidden_dim"],
    num_layers=2,
    bidirectional=True,
    dropout=best_params["dropout"]
).to(device)

[I 2024-12-07 18:40:39,592] A new study created in memory with name: no-name-f8ef5602-2d75-4269-b3c6-30888c159688


Epoch 1/5, Train Loss: 0.6958, Train Accuracy: 0.4896
Epoch 1/5, Validation Accuracy: 0.5643
Epoch 2/5, Train Loss: 0.6916, Train Accuracy: 0.5413
Epoch 2/5, Validation Accuracy: 0.5667
Epoch 3/5, Train Loss: 0.6863, Train Accuracy: 0.5543
Epoch 3/5, Validation Accuracy: 0.5640
Epoch 4/5, Train Loss: 0.6865, Train Accuracy: 0.5582
Epoch 4/5, Validation Accuracy: 0.6077
Epoch 5/5, Train Loss: 0.6823, Train Accuracy: 0.5451
Epoch 5/5, Validation Accuracy: 0.6535
Test Accuracy: 0.6549
Epoch 1/5, Train Loss: 0.6654, Train Accuracy: 0.5802
Epoch 1/5, Validation Accuracy: 0.6373
Epoch 2/5, Train Loss: 0.6483, Train Accuracy: 0.6418
Epoch 2/5, Validation Accuracy: 0.5536
Epoch 3/5, Train Loss: 0.6444, Train Accuracy: 0.6260
Epoch 3/5, Validation Accuracy: 0.6332
Epoch 4/5, Train Loss: 0.6199, Train Accuracy: 0.6399
Epoch 4/5, Validation Accuracy: 0.5725
Epoch 5/5, Train Loss: 0.5943, Train Accuracy: 0.6908
Epoch 5/5, Validation Accuracy: 0.5878
Test Accuracy: 0.5860
Epoch 1/5, Train Loss: 0.6

[I 2024-12-07 18:40:42,842] Trial 0 finished with value: 0.6269411932629175 and parameters: {'lr': 0.00013107659651226693, 'hidden_dim': 212, 'dropout': 0.16050215766531717, 'sequence_length': 7}. Best is trial 0 with value: 0.6269411932629175.


Epoch 5/5, Train Loss: 0.5645, Train Accuracy: 0.7210
Epoch 5/5, Validation Accuracy: 0.7773
Test Accuracy: 0.7782
Trial completed. Mean Validation Accuracy: 0.6269
Epoch 1/5, Train Loss: 0.6951, Train Accuracy: 0.5150
Epoch 1/5, Validation Accuracy: 0.5656
Epoch 2/5, Train Loss: 0.6922, Train Accuracy: 0.5529
Epoch 2/5, Validation Accuracy: 0.5680
Epoch 3/5, Train Loss: 0.6927, Train Accuracy: 0.5384
Epoch 3/5, Validation Accuracy: 0.5655
Epoch 4/5, Train Loss: 0.6896, Train Accuracy: 0.5447
Epoch 4/5, Validation Accuracy: 0.5646
Epoch 5/5, Train Loss: 0.6865, Train Accuracy: 0.5406
Epoch 5/5, Validation Accuracy: 0.5802
Test Accuracy: 0.5774
Epoch 1/5, Train Loss: 0.6627, Train Accuracy: 0.6041
Epoch 1/5, Validation Accuracy: 0.6020
Epoch 2/5, Train Loss: 0.6415, Train Accuracy: 0.6338
Epoch 2/5, Validation Accuracy: 0.5992
Epoch 3/5, Train Loss: 0.6409, Train Accuracy: 0.6184
Epoch 3/5, Validation Accuracy: 0.6138
Epoch 4/5, Train Loss: 0.6090, Train Accuracy: 0.6480
Epoch 4/5, Vali

[I 2024-12-07 18:40:45,300] Trial 1 finished with value: 0.6506421085689379 and parameters: {'lr': 0.000630417256351656, 'hidden_dim': 108, 'dropout': 0.15398920344995168, 'sequence_length': 8}. Best is trial 1 with value: 0.6506421085689379.


...

Epoch 5/5, Train Loss: 0.5895, Train Accuracy: 0.6983
Epoch 5/5, Validation Accuracy: 0.7260
Test Accuracy: 0.7262
Trial completed. Mean Validation Accuracy: 0.6555
Epoch 1/5, Train Loss: 0.6917, Train Accuracy: 0.5393
Epoch 1/5, Validation Accuracy: 0.5664
Epoch 2/5, Train Loss: 0.6901, Train Accuracy: 0.5449
Epoch 2/5, Validation Accuracy: 0.5678
Epoch 3/5, Train Loss: 0.6895, Train Accuracy: 0.5443
Epoch 3/5, Validation Accuracy: 0.5680
Epoch 4/5, Train Loss: 0.6885, Train Accuracy: 0.5478
Epoch 4/5, Validation Accuracy: 0.5664
Epoch 5/5, Train Loss: 0.6897, Train Accuracy: 0.5401
Epoch 5/5, Validation Accuracy: 0.5681
Test Accuracy: 0.5673
Epoch 1/5, Train Loss: 0.6894, Train Accuracy: 0.5390
Epoch 1/5, Validation Accuracy: 0.5516
Epoch 2/5, Train Loss: 0.6880, Train Accuracy: 0.5432
Epoch 2/5, Validation Accuracy: 0.5538
Epoch 3/5, Train Loss: 0.6875, Train Accuracy: 0.5471
Epoch 3/5, Validation Accuracy: 0.5554
Epoch 4/5, Train Loss: 0.6866, Train Accuracy: 0.5513
Epoch 4/5, Vali

[I 2024-12-07 18:41:38,141] Trial 18 finished with value: 0.5549123613631781 and parameters: {'lr': 2.0072300679475442e-05, 'hidden_dim': 101, 'dropout': 0.36746347093362297, 'sequence_length': 6}. Best is trial 13 with value: 0.6676176648517848.


Epoch 4/5, Train Loss: 0.6800, Train Accuracy: 0.5486
Epoch 4/5, Validation Accuracy: 0.6548
Epoch 5/5, Train Loss: 0.6778, Train Accuracy: 0.5678
Epoch 5/5, Validation Accuracy: 0.6552
Test Accuracy: 0.6560
Trial completed. Mean Validation Accuracy: 0.5549
Epoch 1/5, Train Loss: 0.6990, Train Accuracy: 0.4445
Epoch 1/5, Validation Accuracy: 0.5625
Epoch 2/5, Train Loss: 0.6923, Train Accuracy: 0.5419
Epoch 2/5, Validation Accuracy: 0.5671
Epoch 3/5, Train Loss: 0.6899, Train Accuracy: 0.5480
Epoch 3/5, Validation Accuracy: 0.5652
Epoch 4/5, Train Loss: 0.6887, Train Accuracy: 0.5501
Epoch 4/5, Validation Accuracy: 0.5674
Epoch 5/5, Train Loss: 0.6856, Train Accuracy: 0.5425
Epoch 5/5, Validation Accuracy: 0.5644
Test Accuracy: 0.5664
Epoch 1/5, Train Loss: 0.6689, Train Accuracy: 0.5941
Epoch 1/5, Validation Accuracy: 0.5804
Epoch 2/5, Train Loss: 0.6574, Train Accuracy: 0.5932
Epoch 2/5, Validation Accuracy: 0.6172
Epoch 3/5, Train Loss: 0.6302, Train Accuracy: 0.6607
Epoch 3/5, Vali

[I 2024-12-07 18:41:41,479] Trial 19 finished with value: 0.6224365226631142 and parameters: {'lr': 0.0005114261765967908, 'hidden_dim': 205, 'dropout': 0.25962931826456304, 'sequence_length': 7}. Best is trial 13 with value: 0.6676176648517848.


Epoch 5/5, Train Loss: 0.5742, Train Accuracy: 0.7071
Epoch 5/5, Validation Accuracy: 0.7578
Test Accuracy: 0.7540
Trial completed. Mean Validation Accuracy: 0.6224


In [13]:
# Test the fine-tuned model on the internal test set
print("Evaluating the fine-tuned model on the internal test set...")
evaluate_model(fine_tuned_model, internal_test_loaders, device)

Evaluating the fine-tuned model on the internal test set...
Test Accuracy: 0.4766


0.4766355140186916

In [16]:
# Combine all data (train and internal test) for final training
all_train_df = pd.concat([train_df, internal_test_df])

# Split into train and validation loaders for final training
train_split = all_train_df.sample(frac=0.9, random_state=42)
val_split = all_train_df.drop(train_split.index)

all_train_game_loaders = create_game_based_data_loaders(
    train_split, batch_size=32, sequence_length=best_params["sequence_length"]
)
all_val_game_loaders = create_game_based_data_loaders(
    val_split, batch_size=32, sequence_length=best_params["sequence_length"]
)

# Retrain the model on the full dataset
final_model = train_model(
    fine_tuned_model, all_train_game_loaders, all_val_game_loaders, 
    epochs=20, lr=best_params["lr"], device=device
)

Epoch 1/20, Train Loss: 0.6411, Train Accuracy: 0.6310
Epoch 1/20, Validation Accuracy: 0.6202
Epoch 2/20, Train Loss: 0.6285, Train Accuracy: 0.6584
Epoch 2/20, Validation Accuracy: 0.6578
Epoch 3/20, Train Loss: 0.6196, Train Accuracy: 0.6598
Epoch 3/20, Validation Accuracy: 0.5791
Epoch 4/20, Train Loss: 0.6028, Train Accuracy: 0.6630
Epoch 4/20, Validation Accuracy: 0.5623
Epoch 5/20, Train Loss: 0.5775, Train Accuracy: 0.6915
Epoch 5/20, Validation Accuracy: 0.6719
Epoch 6/20, Train Loss: 0.5784, Train Accuracy: 0.6828
Epoch 6/20, Validation Accuracy: 0.6619
Epoch 7/20, Train Loss: 0.5685, Train Accuracy: 0.7009
Epoch 7/20, Validation Accuracy: 0.6615
Epoch 8/20, Train Loss: 0.5678, Train Accuracy: 0.7009
Epoch 8/20, Validation Accuracy: 0.5769
Epoch 9/20, Train Loss: 0.5630, Train Accuracy: 0.7052
Epoch 9/20, Validation Accuracy: 0.6086
Epoch 10/20, Train Loss: 0.5577, Train Accuracy: 0.7107
Epoch 10/20, Validation Accuracy: 0.5769
Epoch 11/20, Train Loss: 0.5480, Train Accuracy:

In [18]:
# Generate predictions for the Kaggle submission
submission_df = predict_for_submission(final_model, test_df, device)
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv.")

Submission file saved as submission.csv.
